<a href="https://colab.research.google.com/github/KristineYW/DS-Unit-2-Applied-Modeling/blob/master/Kristine_Wang_LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model Interpretation
You will use your portfolio project dataset for all assignments this sprint.

Assignment
Complete these tasks for your project, and document your work.

 Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
 Make at least 1 partial dependence plot to explain your model.
 Make at least 1 Shapley force plot to explain an individual prediction.
 Share at least 1 visualization (of any type) on Slack!
If you aren't ready to make these plots with your own dataset, you can practice these objectives with any dataset you've worked with previously. Example solutions are available for Partial Dependence Plots with the Tanzania Waterpumps dataset, and Shapley force plots with the Titanic dataset. (These datasets are available in the data directory of this repository.)

Please be aware that multi-class classification will result in multiple Partial Dependence Plots (one for each class), and multiple sets of Shapley Values (one for each class).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# # Read New York City property sales data
# import pandas as pd
# df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [3]:
# Change column names: replace spaces with underscores

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sc1 = pd.read_csv('https://raw.githubusercontent.com/KristineYW/DS-Unit-2-Applied-Modeling/master/SCDB_2019_01_justiceCentered_LegalProvision_1.csv', encoding = "ISO-8859-1")

sc1.head()

,caseId,docketId,voteId,dateDecision,decisionType,ledCite,term,naturalCourt,chief,docket,...,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,firstAgreement,secondAgreement
0,1946-001,1946-001-01,1946-001-01-01-01-01,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1946-001,1946-001-01,1946-001-01-01-01-02,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1946-001,1946-001-01,1946-001-01-01-01-03,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1946-001,1946-001-01,1946-001-01-01-01-04,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1946-001,1946-001-01,1946-001-01-01-01-05,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sc2 = pd.read_csv('https://raw.githubusercontent.com/KristineYW/DS-Unit-2-Applied-Modeling/master/SCDB_2019_01_justiceCentered_LegalProvision_2.csv', encoding = "ISO-8859-1")

sc2.head()

,ï»¿certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,...,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority
0,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,...,78.0,1,8,1,86,HHBurton,2.0,1.0,1.0,1.0
1,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,...,78.0,1,8,1,84,RHJackson,1.0,1.0,2.0,2.0
2,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,...,78.0,1,8,1,81,WODouglas,1.0,1.0,2.0,2.0
3,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,...,78.0,1,8,1,80,FFrankfurter,4.0,2.0,2.0,2.0
4,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,...,78.0,1,8,1,79,SFReed,1.0,1.0,2.0,2.0


In [5]:
# Join the two large datasets into one dataframe

sc = pd.concat([sc1,sc2], axis=1)

sc.head()

,caseId,docketId,voteId,dateDecision,decisionType,ledCite,term,naturalCourt,chief,docket,...,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority
0,1946-001,1946-001-01,1946-001-01-01-01-01,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,86,HHBurton,2.0,1.0,1.0,1.0
1,1946-001,1946-001-01,1946-001-01-01-01-02,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,84,RHJackson,1.0,1.0,2.0,2.0
2,1946-001,1946-001-01,1946-001-01-01-01-03,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,81,WODouglas,1.0,1.0,2.0,2.0
3,1946-001,1946-001-01,1946-001-01-01-01-04,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,80,FFrankfurter,4.0,2.0,2.0,2.0
4,1946-001,1946-001-01,1946-001-01-01-01-05,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,79,SFReed,1.0,1.0,2.0,2.0


In [6]:
sc = sc.drop(['Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 
              'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 
              'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 
              'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 
              'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 
              'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54'], axis=1)

sc.head()

,caseId,docketId,voteId,dateDecision,decisionType,ledCite,term,naturalCourt,chief,docket,...,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority
0,1946-001,1946-001-01,1946-001-01-01-01-01,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,86,HHBurton,2.0,1.0,1.0,1.0
1,1946-001,1946-001-01,1946-001-01-01-01-02,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,84,RHJackson,1.0,1.0,2.0,2.0
2,1946-001,1946-001-01,1946-001-01-01-01-03,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,81,WODouglas,1.0,1.0,2.0,2.0
3,1946-001,1946-001-01,1946-001-01-01-01-04,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,80,FFrankfurter,4.0,2.0,2.0,2.0
4,1946-001,1946-001-01,1946-001-01-01-01-05,11/18/46,1,91 L. Ed. 3,1946,1301,Vinson,24,...,78.0,1,8,1,79,SFReed,1.0,1.0,2.0,2.0


In [7]:
import datetime

In [8]:
sc.sample(10)

,caseId,docketId,voteId,dateDecision,decisionType,ledCite,term,naturalCourt,chief,docket,...,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority
66152,1979-039,1979-039-01,1979-039-01-01-01-05,2/20/80,1,63 L. Ed. 2d 115,1979,1506,Burger,78-857,...,99.0,1,5,4,99,WEBurger,1.0,1.0,1.0,2.0
101539,1998-077,1998-077-01,1998-077-01-01-01-08,6/17/99,1,144 L. Ed. 2d 319,1998,1607,Rehnquist,98-231,...,102.0,1,5,4,109,RBGinsburg,2.0,2.0,2.0,1.0
117532,2015-005,2015-005-02,2015-005-02-01-01-07,1/20/16,1,193 L. Ed. 2d 535,2015,1704,Roberts,14-450,...,111.0,1,8,1,112,SAAlito,1.0,1.0,1.0,2.0
51013,1972-128,1972-128-01,1972-128-01-01-01-07,5/29/73,1,36 L. Ed. 2d 900,1972,1504,Burger,72-212,...,99.0,1,7,2,100,HABlackmun,3.0,2.0,1.0,2.0
48826,1971-133,1971-133-01,1971-133-01-01-01-07,6/12/72,1,32 L. Ed. 2d 513,1971,1504,Burger,71-322,...,99.0,1,8,1,100,HABlackmun,1.0,1.0,1.0,2.0
56973,1975-044,1975-044-01,1975-044-01-01-01-05,3/3/76,1,47 L. Ed. 2d 196,1975,1506,Burger,74-773,...,99.0,1,6,2,99,WEBurger,3.0,1.0,1.0,2.0
78247,1984-082,1984-082-01,1984-082-01-01-01-04,4/17/85,1,85 L. Ed. 2d 233,1984,1507,Burger,83-1360,...,99.0,1,6,2,99,WEBurger,1.0,1.0,1.0,2.0
113198,2010-006,2010-006-01,2010-006-01-01-01-09,1/11/11,1,178 L. Ed. 2d 588,2010,1704,Roberts,09-837,...,111.0,1,8,0,114,EKagan,NaN,NaN,NaN,NaN
87035,1988-069,1988-069-01,1988-069-01-01-01-07,4/3/89,1,104 L. Ed. 2d 58,1988,1603,Rehnquist,87-453,...,102.0,1,8,0,104,SDOConnor,NaN,NaN,NaN,NaN
103077,2000-022,2000-022-01,2000-022-01-03-01-01,2/21/01,1,148 L. Ed. 2d 866,2000,1607,Rehnquist,99-1240,...,102.0,1,5,4,102,WHRehnquist,1.0,2.0,1.0,2.0


In [9]:
# Change dateDecision to datetime data type 


def yearConvert(date):
    year_int = int(date[-2:])
    if year_int > 20:
        edited_str = date[:-2] + '19' + date[-2:]
    else:
        edited_str = date[:-2] + '20' + date[-2:]
    return(pd.to_datetime(edited_str))
sc['dateDecision'] = [yearConvert(i) for i in sc['dateDecision']]
sc['dateDecision'].describe

<bound method NDFrame.describe of 0        1946-11-18
1        1946-11-18
2        1946-11-18
3        1946-11-18
4        1946-11-18
            ...    
120553   2019-05-28
120554   2019-05-28
120555   2019-05-28
120556   2019-05-28
120557   2019-05-28
Name: dateDecision, Length: 120558, dtype: datetime64[ns]>

In [10]:
# Introduce new features for senate majority party and presidential party


sc['partyOfPresident'] = np.where((sc['dateDecision'] < '1953-01-19') | ('1961-01-20' < sc['dateDecision']) & (sc['dateDecision'] < '1969-01-19') | 
                                  ('1977-01-20' < sc['dateDecision']) & (sc['dateDecision'] < '1981-01-19') | 
                                  ('1993-01-20' < sc['dateDecision']) & (sc['dateDecision'] < '2001-01-19') | 
                                  ('2009-01-20' < sc['dateDecision']) & (sc['dateDecision'] < '2017-01-20'), 
                                  'Democrat','Republican')

In [11]:
sc['partyOfAppointing'] = np.where((sc['justiceName'] == 'BRWhite') | (sc['justiceName'] == 'WODouglas') | (sc['justiceName'] == 'Tmarshall') | 
                                (sc['justiceName'] == 'HLBlack') | (sc['justiceName'] == 'TCClark') | (sc['justiceName'] == 'Ffrankfurter') | 
                                (sc['justiceName']== 'RBGinsburg') | (sc['justiceName'] == 'SGBreyer') | (sc['justiceName'] == 'HHBurton') | 
                                (sc['justiceName'] == 'SFReed') | (sc['justiceName'] == 'RHJackson') | (sc['justiceName'] == 'FMVinson') | 
                                (sc['justiceName'] == 'Sminton') | (sc['justiceName'] == 'Ssotomayor') | (sc['justiceName'] == 'Afortas') | 
                                (sc['justiceName'] == 'Ekagan') | (sc['justiceName'] == 'Fmurphy') | (sc['justiceName'] == 'WBRutledge') | 
                                (sc['justiceName'] == 'AJGoldberg'), 'Democrat','Republican' )

In [12]:
sc['partyOfSenate'] = np.where((sc['dateDecision'] < '1947-01-03') | ('1949-01-03' < sc['dateDecision']) & (sc['dateDecision'] < '1953-01-02') | 
                                  ('1955-01-03' < sc['dateDecision']) & (sc['dateDecision'] < '1981-01-02') | 
                                  ('1987-01-03' < sc['dateDecision']) & (sc['dateDecision'] < '1995-01-02') |
                                  ('2001-01-03' < sc['dateDecision']) & (sc['dateDecision'] < '2002-11-11') | 
                                  ('2007-01-03' < sc['dateDecision']) & (sc['dateDecision'] < '2017-01-02'),
                                  'Democrat','Republican')

sc['partyOfSenate'].value_counts(normalize=True)

Democrat      0.712404
Republican    0.287596
Name: partyOfSenate, dtype: float64

In [13]:
pd.set_option('display.max_columns',100)
    
sc.head(10)

,caseId,docketId,voteId,dateDecision,decisionType,ledCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,firstAgreement,secondAgreement,ï»¿certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,partyOfPresident,partyOfAppointing,partyOfSenate
0,1946-001,1946-001-01,1946-001-01-01-01-01,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,86,HHBurton,2.0,1.0,1.0,1.0,Democrat,Democrat,Democrat
1,1946-001,1946-001-01,1946-001-01-01-01-02,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,84,RHJackson,1.0,1.0,2.0,2.0,Democrat,Democrat,Democrat
2,1946-001,1946-001-01,1946-001-01-01-01-03,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,81,WODouglas,1.0,1.0,2.0,2.0,Democrat,Democrat,Democrat
3,1946-001,1946-001-01,1946-001-01-01-01-04,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,80,FFrankfurter,4.0,2.0,2.0,2.0,Democrat,Republican,Democrat
4,1946-001,1946-001-01,1946-001-01-01-01-05,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,79,SFReed,1.0,1.0,2.0,2.0,Democrat,Democrat,Democrat
5,1946-001,1946-001-01,1946-001-01-01-01-06,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,78,HLBlack,1.0,2.0,2.0,2.0,Democrat,Democrat,Democrat
6,1946-001,1946-001-01,1946-001-01-01-01-07,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,85,WBRutledge,1.0,1.0,2.0,2.0,Democrat,Democrat,Democrat
7,1946-001,1946-001-01,1946-001-01-01-01-08,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,NaN,NaN,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. Ã 33,78.0,78.0,1,8,1,82,FMurphy,1.0,1.0,2.0,2.0,Democrat,Republican,Democrat
8,1946-001,1946-001-01,1946-001-01-01-01-09,1946-11-18,1,91 L. Ed. 3,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/46,10/23/46,198.0,NaN,172.0,NaN,6,NaN

In [14]:
sc['term'].value_counts()

1967    2943
1986    2583
1946    2556
1975    2496
1973    2448
        ... 
2017     801
2007     790
2014     783
2016     747
2018     720
Name: term, Length: 73, dtype: int64

In [15]:
# Retrain features to be categorical for OneHotEncoding

all_columns = list(sc)
sc[all_columns] = sc[all_columns].astype(str)

In [16]:
sc['direction'].value_counts(normalize=True)

2.0    0.496666
1.0    0.447419
nan    0.055915
Name: direction, dtype: float64

In [17]:
sc['direction'] = sc['direction'].replace({'nan':'-1'})

In [18]:
sc['direction'].value_counts(normalize=True)

2.0    0.496666
1.0    0.447419
-1     0.055915
Name: direction, dtype: float64

In [19]:
sc['partyOfPresident'].value_counts(normalize=True)

Republican    0.551909
Democrat      0.448091
Name: partyOfPresident, dtype: float64

In [20]:
sc['partyOfSenate'].value_counts(normalize=True)

Democrat      0.712404
Republican    0.287596
Name: partyOfSenate, dtype: float64

In [21]:
sc = sc.fillna(0)

In [22]:
# Drop all outcome variables

sc = sc.drop(['decisionDirection', 'partyWinning','majVotes','minVotes','majority','decisionType','declarationUncon',
              'caseDispositionUnusual','precedentAlteration','voteUnclear','majOpinWriter','majOpinAssigner',
              'splitVote','vote','opinion','majority','firstAgreement','secondAgreement'], axis=1)

In [23]:
# Split the dataframe into train and test sets 

from sklearn.model_selection import train_test_split

train, test = train_test_split(sc, train_size=0.9, test_size=0.10, 
                              stratify=sc['direction'], random_state=42)

In [24]:
train, val = train_test_split(train, train_size=0.85, test_size=0.15, 
                              stratify=train['direction'], random_state=42)

In [25]:
train.shape, val.shape, test.shape

((92226, 43), (16276, 43), (12056, 43))

In [26]:
count = sc[['justiceName','direction']]


count.head()

,justiceName,direction
0,HHBurton,1.0
1,RHJackson,2.0
2,WODouglas,2.0
3,FFrankfurter,2.0
4,SFReed,2.0


In [46]:
count['justiceName'].value_counts()

WJBrennan       7837
BRWhite         7199
WHRehnquist     6442
JPStevens       5928
WODouglas       5816
HABlackmun      5421
TMarshall       5384
PStewart        5278
HLBlack         5060
SDOConnor       4104
WEBurger        4103
AScalia         3900
AMKennedy       3744
LFPowell        3654
JHarlan2        3401
TCClark         3269
EWarren         3267
CThomas         3103
FFrankfurter    2830
RBGinsburg      2807
SGBreyer        2644
DHSouter        2361
HHBurton        2250
SFReed          1840
FMVinson        1378
RHJackson       1377
JGRoberts       1308
SAAlito         1258
SMinton         1070
CEWhittaker      980
SSotomayor       909
AFortas          836
EKagan           759
WBRutledge       698
FMurphy          685
AJGoldberg       674
NMGorsuch        173
BMKavanaugh       70
Name: justiceName, dtype: int64

In [27]:
count = count[count['direction'] != '-1']

count.head()

,justiceName,direction
0,HHBurton,1.0
1,RHJackson,2.0
2,WODouglas,2.0
3,FFrankfurter,2.0
4,SFReed,2.0


In [28]:
count.shape

(113817, 2)

In [29]:
# count['totalVotes'] = count.groupby(['justiceName','direction']).transform(np.size)

count2 = count.groupby(['justiceName','direction']).size().reset_index(name='totalVotes')
count2 = count2.replace({'1.0': 'Conservative', '2.0': 'Liberal'})

count2

,justiceName,direction,totalVotes
0,AFortas,Conservative,255
1,AFortas,Liberal,581
2,AJGoldberg,Conservative,162
3,AJGoldberg,Liberal,512
4,AMKennedy,Conservative,2200
...,...,...,...
71,WHRehnquist,Liberal,1895
72,WJBrennan,Conservative,2311
73,WJBrennan,Liberal,5526
74,WODouglas,Conservative,1265


In [47]:
count2['justiceName'].value_counts()

HLBlack         2
AMKennedy       2
CThomas         2
WBRutledge      2
WEBurger        2
HHBurton        2
CEWhittaker     2
JGRoberts       2
WHRehnquist     2
WJBrennan       2
AJGoldberg      2
EKagan          2
AFortas         2
BMKavanaugh     2
LFPowell        2
TMarshall       2
RBGinsburg      2
DHSouter        2
FMurphy         2
SDOConnor       2
HABlackmun      2
SGBreyer        2
EWarren         2
BRWhite         2
AScalia         2
FMVinson        2
SFReed          2
JHarlan2        2
TCClark         2
JPStevens       2
SSotomayor      2
SAAlito         2
SMinton         2
WODouglas       2
RHJackson       2
NMGorsuch       2
PStewart        2
FFrankfurter    2
Name: justiceName, dtype: int64

In [81]:
order = ['Liberal','Conservative']

count_p = pd.pivot_table(
    count2,
    index='justiceName',
    columns='direction',
    values='totalVotes'
)

count_p = count_p[order]

count_p['Liberal'] = count_p['Liberal'] * -1

In [83]:
count_p.head()

direction,Liberal,Conservative
justiceName,,
AFortas,-581,255
AJGoldberg,-512,162
AMKennedy,-1544,2200
AScalia,-1309,2591
BMKavanaugh,-27,43


In [104]:
# sort by desired column
import plotly.graph_objects as go

count_p = count_p.sort_values(by='Liberal', ascending = False)

fig = go.Figure()

for column in count_p.columns:
    fig.add_trace(go.Bar(
        x = count_p[column],
        y = count_p.index,
        name = column,
        orientation = 'h',
    ))

fig.update_layout(
    autosize=False,
    width=700,
    height=1000,
    barmode = 'relative',
    title = 'Totaled Liberal/Conservative Votes by the Supreme Court Justices',
    yaxis = dict(
        tickmode = 'array',
        tickvals = ['BMKavanaugh', 'NMGorsuch', 'SAAlito', 'CEWhittaker', 'SMinton', 'EKagan',
                    'AJGoldberg','RHJackson','FMurphy','WBRutledge','JGRoberts','SSotomayor','AFortas',
                    'FMVinson','SFReed','CThomas','HHBurton','FFrankfurter','AScalia','DHSouter', 
                    'WEBurger', 'LFPowell', 'JHarlan2', 'AMKennedy', 'SGBreyer','SDOConnor','TCClark',
                    'RBGinsburg','WHRehnquist','EWarren','PStewart','HABlackmun','BRWhite','JPStevens',
                    'HLBlack','TMarshall','WODouglas','WJBrennan'],
        ticktext = ['Kavanaugh ', 'Gorsuch ', 'Alito ', 'Whittaker ', 'Minton ', 'Kagan ',
                    'Goldberg ','Jackson ','Murphy ','Rutledge ','Roberts ','Sotomayor ','Fortas ',
                    'Vinson ','Reed ','Thomas ','Burton ','Frankfurter ','Scalia ','Souter ', 
                    'Burger ', 'Powell ', 'Harlan2 ', 'Kennedy ', 'Breyer ','OConnor ','Clark ',
                    'Ginsburg ','Rehnquist ','Warren ','Stewart ','Blackmun ','White ','Stevens ',
                    'Black ','Marshall ','Douglas ','Brennan ']),
    legend=dict(
        x=.71,
        y=0.01,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=16,
            color="black"
        ),
        bgcolor="white",
        bordercolor="gray",
        borderwidth=2)
)


fig.show()